In [2]:
import os
#import shutil
import pandas as pd
import gradio as gr
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
#import seaborn as sns
from pylab import rcParams

import matplotlib.pyplot as plt
tf.get_logger().setLevel('ERROR')

#sns.set(style='whitegrid', palette='muted', font_scale=1.2)
HAPPY_COLORS_PALETTE = ["#01BEFE", "#FFDD00", "#FF7D00", "#FF006D", "#ADFF02", "#8F00FF"]
#sns.set_palette(sns.color_palette(HAPPY_COLORS_PALETTE))
rcParams['figure.figsize'] = 12, 8
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split
from tensorflow import keras

In [3]:
#!pip install sklearn

In [4]:
#!pip install gradio

In [5]:
#!pip install gradio

In [6]:
traindf = pd.read_csv('train.csv',encoding = 'utf-8')
testdf = pd.read_csv('test.csv',encoding = 'utf-8')
validdf = pd.read_csv('validation.csv',encoding = 'utf-8')

In [7]:
traindf.head()

,text,intent
0,can you help me to create an account,CreateAccount
1,User call to check claims status,CheckStatus
2,Caller had questions about the provider relief...,CallForInformation
3,have their EHB account enabled and password reset,UnlockAccount
4,I need assistance with creating an EHB account,CreateEHBAccount


In [9]:
trainfeatures = traindf.copy()
trainlabels = trainfeatures.pop('intent')

In [10]:
trainlabels

0           CreateAccount
1             CheckStatus
2      CallForInformation
3           UnlockAccount
4        CreateEHBAccount
              ...        
588           CheckStatus
589         FileComplaint
590      FindHealthCenter
591           MakeChanges
592         AccessAccount
Name: intent, Length: 593, dtype: object

In [11]:
trainfeatures=trainfeatures.values

In [13]:
#trainfeatures

In [14]:
from sklearn.preprocessing import LabelBinarizer

In [15]:
binarizer = LabelBinarizer()
trainlabels = binarizer.fit_transform(trainlabels.values)

In [17]:
trainlabels

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]])

In [18]:
trainlabels.shape

(593, 33)

In [19]:
testfeatures=testdf.copy()
testlabels=testfeatures.pop("intent")
validfeatures=validdf.copy()
validlabels=validfeatures.pop("intent")

testfeatures=testfeatures.values
validfeatures=validfeatures.values

testlabels=binarizer.transform(testlabels.values)
validlabels=binarizer.transform(validlabels.values)

In [15]:
bert_model_name = 'small_bert/bert_en_uncased_L-8_H-512_A-8' 
map_name_to_handle = {
    'bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3',
    'bert_en_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_cased_L-12_H-768_A-12/3',
    'bert_multi_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/3',
    'small_bert/bert_en_uncased_L-2_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-2_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-2_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-2_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-4_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-4_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-4_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-4_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-6_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-6_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-6_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-6_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-8_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-8_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-8_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-8_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-10_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-10_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-10_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-10_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-12_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-12_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-12_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-768_A-12/1',
    'albert_en_base':
        'https://tfhub.dev/tensorflow/albert_en_base/2',
    'electra_small':
        'https://tfhub.dev/google/electra_small/2',
    'electra_base':
        'https://tfhub.dev/google/electra_base/2',
    'experts_pubmed':
        'https://tfhub.dev/google/experts/bert/pubmed/2',
    'experts_wiki_books':
        'https://tfhub.dev/google/experts/bert/wiki_books/2',
    'talking-heads_base':
        'https://tfhub.dev/tensorflow/talkheads_ggelu_bert_en_base/1',
}

map_model_to_preprocess = {
    'bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/2',
    'bert_en_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_cased_preprocess/2',
    'small_bert/bert_en_uncased_L-2_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/2',
    'small_bert/bert_en_uncased_L-2_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/2',
    'small_bert/bert_en_uncased_L-2_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/2',
    'small_bert/bert_en_uncased_L-2_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/2',
    'small_bert/bert_en_uncased_L-4_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/2',
    'small_bert/bert_en_uncased_L-4_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/2',
    'small_bert/bert_en_uncased_L-4_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/2',
    'small_bert/bert_en_uncased_L-4_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/2',
    'small_bert/bert_en_uncased_L-6_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/2',
    'small_bert/bert_en_uncased_L-6_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/2',
    'small_bert/bert_en_uncased_L-6_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/2',
    'small_bert/bert_en_uncased_L-6_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/2',
    'small_bert/bert_en_uncased_L-8_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/2',
    'small_bert/bert_en_uncased_L-8_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/2',
    'small_bert/bert_en_uncased_L-8_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/2',
    'small_bert/bert_en_uncased_L-8_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/2',
    'small_bert/bert_en_uncased_L-10_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/2',
    'small_bert/bert_en_uncased_L-10_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/2',
    'small_bert/bert_en_uncased_L-10_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/2',
    'small_bert/bert_en_uncased_L-10_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/2',
    'small_bert/bert_en_uncased_L-12_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/2',
    'small_bert/bert_en_uncased_L-12_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/2',
    'small_bert/bert_en_uncased_L-12_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/2',
    'small_bert/bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/2',
    'bert_multi_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_multi_cased_preprocess/2',
    'albert_en_base':
        'https://tfhub.dev/tensorflow/albert_en_preprocess/2',
    'electra_small':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/2',
    'electra_base':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/2',
    'experts_pubmed':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/2',
    'experts_wiki_books':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/2',
    'talking-heads_base':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/2',
}

tfhub_handle_encoder = map_name_to_handle[bert_model_name]
tfhub_handle_preprocess = map_model_to_preprocess[bert_model_name]

print(f'BERT model selected           : {tfhub_handle_encoder}')
print(f'Preprocess model auto-selected: {tfhub_handle_preprocess}')

BERT model selected           : https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-512_A-8/1
Preprocess model auto-selected: https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/2


In [16]:
bert_preprocess_model = hub.KerasLayer(tfhub_handle_preprocess)

In [17]:
trainfeatures[0]

array(['can you help me to create an account'], dtype=object)

In [1]:
text_test = trainfeatures[0]
text_preprocessed = bert_preprocess_model(text_test)

print(f'Keys       : {list(text_preprocessed.keys())}')
print(f'Shape      : {text_preprocessed["input_word_ids"].shape}')
print(f'Word Ids   : {text_preprocessed["input_word_ids"][0, :12]}')
print(f'Input Mask : {text_preprocessed["input_mask"][0, :12]}')
print(f'Type Ids   : {text_preprocessed["input_type_ids"][0, :12]}')

NameError: name 'trainfeatures' is not defined

In [19]:
bert_model = hub.KerasLayer(tfhub_handle_encoder)

In [20]:
bert_results = bert_model(text_preprocessed)

print(f'Loaded BERT: {tfhub_handle_encoder}')
print(f'Pooled Outputs Shape:{bert_results["pooled_output"].shape}')
print(f'Pooled Outputs Values:{bert_results["pooled_output"][0, :12]}')
print(f'Sequence Outputs Shape:{bert_results["sequence_output"].shape}')
print(f'Sequence Outputs Values:{bert_results["sequence_output"][0, :12]}')

Loaded BERT: https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-512_A-8/1
Pooled Outputs Shape:(1, 512)
Pooled Outputs Values:[-0.0216167  -0.04185808 -0.999985    0.2675273   0.8037594  -0.19578242
 -0.99088526 -0.24350508  0.31525406 -0.16903779 -0.78438044 -0.19907   ]
Sequence Outputs Shape:(1, 128, 512)
Sequence Outputs Values:[[-0.31405294 -0.01957725  1.5631087  ... -0.02379022 -0.12260876
   0.74026775]
 [ 0.06647187  0.35380468  0.6203449  ...  0.01576915  0.85667586
   0.48369995]
 [-0.90434897 -0.25116926  1.407662   ...  0.2007662  -0.84128666
   0.44172603]
 ...
 [ 0.35363457  0.00938005  1.9383132  ...  0.02902699 -0.05458811
   0.9358662 ]
 [ 0.02808828 -0.18320957  0.99557954 ...  0.28162864 -0.12516196
   0.23036662]
 [ 0.0061673  -0.07437679  0.94025993 ...  0.30151168 -0.13263541
   0.24854773]]


In [21]:
type(trainlabels.shape[1])

int

In [22]:
def build_classifier_model():
    text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
    preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess, name='preprocessing')
    encoder_inputs = preprocessing_layer(text_input)
    encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=True, name='BERT_encoder')
    outputs = encoder(encoder_inputs)
    net = outputs['pooled_output']
    net = tf.keras.layers.Dropout(0.1)(net)
    net = tf.keras.layers.Dense(trainlabels.shape[1], activation=None, name='classifier')(net)
    return tf.keras.Model(text_input, net)

In [23]:
classifier_model = build_classifier_model()
bert_raw_result = classifier_model(tf.constant(trainfeatures[0]))
print(tf.keras.activations.softmax(bert_raw_result))

tf.Tensor(
[[0.05209855 0.01065828 0.09195076 0.01656543 0.01953142 0.0088785
  0.01741078 0.00443179 0.0244157  0.01634551 0.03970407 0.02189673
  0.00368175 0.03056744 0.02434217 0.0321109  0.01416053 0.02666217
  0.01871175 0.01800307 0.03878236 0.08130637 0.02061501 0.025997
  0.0739853  0.02573029 0.08293378 0.02238583 0.03295208 0.05127613
  0.02206629 0.01199732 0.01784496]], shape=(1, 33), dtype=float32)


In [24]:
classifier_model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text (InputLayer)               [(None,)]            0                                            
__________________________________________________________________________________________________
preprocessing (KerasLayer)      {'input_type_ids': ( 0           text[0][0]                       
__________________________________________________________________________________________________
BERT_encoder (KerasLayer)       {'sequence_output':  41373185    preprocessing[0][0]              
                                                                 preprocessing[0][1]              
                                                                 preprocessing[0][2]              
______________________________________________________________________________________________

In [25]:
loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
metrics = tf.metrics.CategoricalAccuracy()

In [26]:
epochs=10
optimizer=tf.keras.optimizers.Adam(1e-5)
classifier_model.compile(optimizer=optimizer,
                         loss=loss,
                         metrics=metrics)

In [27]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

In [28]:
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

In [29]:
print(f'Training model with {tfhub_handle_encoder}')
history = classifier_model.fit(x=trainfeatures,y=trainlabels,
                               validation_data=(validfeatures,validlabels),
                               batch_size=32,
                               epochs=epochs,
                              callbacks=[cp_callback])

Training model with https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-512_A-8/1
Epoch 1/10
19/19 [==============================] - 195s 9s/step - loss: 3.5556 - categorical_accuracy: 0.0489 - val_loss: 3.4866 - val_categorical_accuracy: 0.0000e+00

Epoch 00001: saving model to training_1\cp.ckpt
Epoch 2/10
19/19 [==============================] - 166s 9s/step - loss: 3.1966 - categorical_accuracy: 0.1164 - val_loss: 3.1557 - val_categorical_accuracy: 0.1000

Epoch 00002: saving model to training_1\cp.ckpt
Epoch 3/10
19/19 [==============================] - 176s 9s/step - loss: 2.8973 - categorical_accuracy: 0.2782 - val_loss: 3.0554 - val_categorical_accuracy: 0.2000

Epoch 00003: saving model to training_1\cp.ckpt
Epoch 4/10
19/19 [==============================] - 178s 9s/step - loss: 2.5900 - categorical_accuracy: 0.3828 - val_loss: 2.5637 - val_categorical_accuracy: 0.5000

Epoch 00004: saving model to training_1\cp.ckpt
Epoch 5/10
19/19 [==============================

In [31]:
classifier_model.save_weights('bert_weights_17_Nov_1630hrs.h5')

In [286]:
def print_my_examples(inputs, results,confidence_score):
    result_for_printing = \
    [f'input: {inputs[i]:<30} : intent_classified: {results[i]}'
                         for i in range(len(inputs))]
    print(*result_for_printing, sep='\n')
    print(results)
    print(confidence_score)


examples = [
    "I am not able to apply grants"
    
]


results = tf.nn.softmax(classifier_model(tf.constant(examples)))
#print(results)
confidence_score = tf.reduce_max(results, axis=-1).numpy()
intents=binarizer.inverse_transform(results.numpy())




In [109]:
def print_my_examples_loaded(description,classifier_model):
    result = tf.nn.softmax(classifier_model(tf.constant([description])))
    confidence_score = tf.reduce_max(result, axis=-1).numpy()
    print(confidence_score[0])
    if confidence_score[0] < 0.15:
        return "Unidentified Intent"
    else:
        intents=binarizer.inverse_transform(result.numpy())
        return intents[0]


In [112]:
description = "I am not able to reset password"

In [113]:
print_my_examples_loaded(description,classifier_model)

0.7685829


'ResetPassword'

In [ ]:
def CreateModel():
    print('Create Model')

In [114]:
classifier_model.save("ModelSave/BERTImprovedPerf")

In [116]:
loaded_model = keras.models.load_model("ModelSave/BERTImprovedPerf")

In [117]:
def print_my_examples_loaded(description,loaded_model):
    result = tf.nn.softmax(loaded_model(tf.constant([description])))
    confidence_score = tf.reduce_max(result, axis=-1).numpy()
    print(confidence_score[0])
    if confidence_score[0] < 0.15:
        return "Unidentified Intent"
    else:
        intents=binarizer.inverse_transform(result.numpy())
        return intents[0]

In [118]:
description = "I am not able to reset password"

In [119]:
print_my_examples_loaded(description,loaded_model)

0.7685829


'ResetPassword'

In [106]:
print_my_examples(examples, intents,confidence_score)

NameError: name 'print_my_examples' is not defined

In [105]:
results[0]

NameError: name 'results' is not defined

In [289]:
classifier_model.save_weights('bert_weights_17_Nov_Accuracy90.h5')

In [296]:
file_path_model = r"C:\DMIProjects\HRSA\intentutterencesclassification\notebooks\Model_weights\Accuracy\17Nov\bert_weights_16_Nov_Accuracy90.h5"

In [120]:
def print_my_examples_loaded(description,loaded_model):
    result = tf.nn.softmax(loaded_model(tf.constant([description])))
    confidence_score = tf.reduce_max(result, axis=-1).numpy()
    print(confidence_score[0])
    if confidence_score[0] < 0.15:
        return "Unidentified Intent"
    else:
        intents=binarizer.inverse_transform(result.numpy())
        return intents[0]


#examples = "I am not able to apply grants"
    



# results = tf.nn.softmax(model_loaded(tf.constant(examples)))
# #print(results)
# confidence_score = tf.reduce_max(results, axis=-1).numpy()
# intents=binarizer.inverse_transform(results.numpy())

In [121]:
def intent_values(description):
    intent = print_my_examples_loaded(description,loaded_model)
    return intent


In [122]:
demo = gr.Interface(fn=intent_values, inputs="text", outputs="text")
demo.launch(share = True) 

Running on local URL:  http://127.0.0.1:7869
Running on public URL: https://38e1676bfa0013b9.gradio.app

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


(<gradio.routes.App at 0x2401fbc4520>,
 'http://127.0.0.1:7869/',
 'https://38e1676bfa0013b9.gradio.app')

0.7354102
0.1270802
0.1270802
0.435295
0.18587995
0.69970983
0.69970983
0.69970983
0.08441927
0.69970983
0.124393806
0.16607645
0.124393806
0.16607645
0.13965459
0.13379951
0.11362834
0.6651249
0.16510715
0.2474255
0.12239976
0.17754807
0.3545213
0.27670768
0.17591144
0.17016336
0.63362473
0.47535393
0.17016336
0.15201701


ERROR:paramiko.transport:Socket exception: An existing connection was forcibly closed by the remote host (10054)


In [259]:
tf.reduce_max(results, axis=-1).numpy()[0]

0.5986389

In [4]:
binarizer.classes_

NameError: name 'binarizer' is not defined

In [231]:
help(binarizer.inverse_transform)

Help on method inverse_transform in module sklearn.preprocessing._label:

inverse_transform(Y, threshold=None) method of sklearn.preprocessing._label.LabelBinarizer instance
    Transform binary labels back to multi-class labels.
    
    Parameters
    ----------
    Y : {ndarray, sparse matrix} of shape (n_samples, n_classes)
        Target values. All sparse matrices are converted to CSR before
        inverse transformation.
    
    threshold : float, default=None
        Threshold used in the binary and multi-label cases.
    
        Use 0 when ``Y`` contains the output of decision_function
        (classifier).
        Use 0.5 when ``Y`` contains the output of predict_proba.
    
        If None, the threshold is assumed to be half way between
        neg_label and pos_label.
    
    Returns
    -------
    y : {ndarray, sparse matrix} of shape (n_samples,)
        Target values. Sparse matrix will be of CSR format.
    
    Notes
    -----
    In the case when the binary labe

In [229]:
binarizer.inverse_transform()

TypeError: inverse_transform() missing 1 required positional argument: 'Y'

In [224]:
intents=binarizer.inverse_transform(results.numpy())

In [225]:
print(intents)

['ResetPassword']


In [219]:
print_my_examples(examples, intents)

input: I need assistance in resetting my EHB password                                                       : estimated intent: ResetPassword
['ResetPassword']



In [211]:
res = classifier_model.predict(examples)

In [212]:
confidence_score = tf.nn.softmax(res,axis = -1)

In [213]:
confidence_score

<tf.Tensor: shape=(1, 33), dtype=float32, numpy=
array([[2.4068968e-03, 6.6007175e-03, 5.7874951e-03, 4.8948722e-03,
        3.7059437e-03, 7.5100744e-03, 1.9509005e-03, 1.7490827e-03,
        1.2967455e-02, 1.2455342e-03, 4.9467487e-03, 5.5766879e-03,
        1.5188695e-03, 1.1028739e-03, 1.5725570e-03, 2.2570104e-03,
        3.8127701e-03, 1.1771732e-03, 2.7596564e-03, 7.4575236e-03,
        8.6439140e-03, 6.8460195e-03, 6.8918876e-03, 3.2903636e-03,
        2.5876418e-03, 7.5198855e-04, 3.4503567e-03, 1.5553408e-02,
        7.5348746e-03, 8.1114799e-01, 9.0396247e-04, 7.9665193e-03,
        4.3430179e-02]], dtype=float32)>

In [214]:
a = confidence_score
tf.reduce_max(a, axis=1).numpy()

array([0.811148], dtype=float32)

In [41]:
import pickle


In [42]:
file_name = 'BERTModelSave.sav'

In [44]:
history

In [47]:
import torch
torch.save(classifier_model,file_name)

AttributeError: Can't pickle local object 'Loader._recreate_base_user_object.<locals>._UserObject'

In [ ]:
#saved_model = torch.load('path/to/model')

In [46]:
classifier_model.save_pretrained(file_name)


AttributeError: 'Functional' object has no attribute 'save_pretrained'

In [3]:
def intent_values(description):
    


ModuleNotFoundError: No module named 'gradio'